# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2023-05-01 17:38:17,565 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

qt.dbus.integration: Could not connect "org.freedesktop.IBus" to globalEngineChanged(QString)


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [3]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self, num):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        
        if num == 1:
            return (5 * 0.3048, -3 * 0.3048, 0)
        elif num == 2:
            return (5 * 0.3048, 3 * 0.3048, 0)
        elif num == 3:
            return (0, 3 * 0.3048, 0)
        else:
            return (-3 * 0.3048, -2 * 0.3048, 0)
        
        # raise NotImplementedError("get_pose is not implemented")

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        sensor_ranges = np.zeros(18)
        sensor_bearings = np.zeros(18)
        
        # start
        ble.send_command(CMD.SEND_TWO_INTS, "")
        
        while sensor_ranges[0] == 0:
            # wait for readings
            string_value = ble.receive_string(ble.uuid['RX_STRING'])
            if string_value != None:
                print(string_value)
                data = string_value.split('|')
                for s in data:
                    if s != '':
                        s = s.split(',')
                        tof = int(s[0])
                        idx = int(s[1])
                        if sensor_ranges[idx] == 0:
                            sensor_ranges[idx] = tof
            
            print(sensor_ranges)
            
        # stop
        ble.send_command(CMD.SEND_THREE_FLOATS, "")
                            

        
        return np.array(sensor_ranges)[np.newaxis].T / 1000, np.array(sensor_bearings)[np.newaxis].T




        
        


In [4]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2023-05-01 17:38:21,299 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:b1:6a:38:3C
2023-05-01 17:38:24,558 | INFO     |: Connected to C0:83:b1:6a:38:3C


/home/zhongqi_tao/ece-5160/Simulator/FastRobots-sim-release-release-mirror/notebooks/base_ble.py:87: FutureWarning: This method will be removed future version, pass the callback to the BleakClient constructor instead.
  self.client.set_disconnected_callback(self.disconnect_handler)


In [5]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2023-05-01 17:38:26,859 | INFO     |:  | Number of observations per grid cell: 18
2023-05-01 17:38:26,860 | INFO     |:  | Precaching Views...


/home/zhongqi_tao/ece-5160/Simulator/FastRobots-sim-release-release-mirror/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2023-05-01 17:38:28,710 | INFO     |:  | Precaching Time: 1.850 secs
2023-05-01 17:38:28,711 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-01 17:38:28,712 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [9]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
current_gt = robot.get_pose(1)
cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2023-05-01 17:29:04,968 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-01 17:29:04,969 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
457,0|642,1|2496,2|1099,3|1191,4|2714,5|3116,6|1372,7|481,8|346,9|285,10|302,11|387,12|504,13|428,14|463,15|602,16|1064,17|
[ 457.  642. 2496. 1099. 1191. 2714. 3116. 1372.  481.  346.  285.  302.
  387.  504.  428.  463.  602. 1064.]
2023-05-01 17:29:13,879 | INFO     |: Update Step
2023-05-01 17:29:13,881 | INFO     |:      | Update Time: 0.002 secs
2023-05-01 17:29:13,881 | INFO     |: Bel index     : (10, 1, 11) with prob = 0.9707422
2023-05-01 17:29:13,882 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2023-05-01 17:29:13,883 | INFO     |: Belief        : (1.524, -0.914, 50.000)


In [10]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
current_gt = robot.get_pose(2)
cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2023-05-01 17:30:50,099 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-01 17:30:50,100 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
441,0|552,1|419,2|363,3|397,4|561,5|996,6|2402,7|386,8|202,9|406,10|2395,11|1438,12|661,13|468,14|403,15|415,16|508,17|
[ 441.  552.  419.  363.  397.  561.  996. 2402.  386.  202.  406. 2395.
 1438.  661.  468.  403.  415.  508.]
2023-05-01 17:30:58,956 | INFO     |: Update Step
2023-05-01 17:30:58,958 | INFO     |:      | Update Time: 0.001 secs
2023-05-01 17:30:58,958 | INFO     |: Bel index     : (11, 6, 10) with prob = 0.9999999
2023-05-01 17:30:58,959 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2023-05-01 17:30:58,960 | INFO     |: Belief        : (1.829, 0.610, 30.000)


In [6]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
current_gt = robot.get_pose(3)
cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2023-05-01 17:38:35,755 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-01 17:38:35,756 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
841,0|465,1|343,2|329,3|428,4|739,5|713,6|775,7|2368,8|2079,9|2186,10|996,11|749,12|1971,13|892,14|520,15|375,16|353,17|
[ 841.  465.  343.  329.  428.  739.  713.  775. 2368. 2079. 2186.  996.
  749. 1971.  892.  520.  375.  353.]
2023-05-01 17:38:44,622 | INFO     |: Update Step
2023-05-01 17:38:44,624 | INFO     |:      | Update Time: 0.002 secs
2023-05-01 17:38:44,625 | INFO     |: Bel index     : (5, 8, 13) with prob = 0.9999999
2023-05-01 17:38:44,626 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2023-05-01 17:38:44,626 | INFO     |: Belief        : (0.000, 1.219, 90.000)
2023-05-01 17:39:15,304 | INFO     |: Disconnected from C0:83:B1:6A:38:3C


In [8]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
current_gt = robot.get_pose(4)
cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2023-05-01 17:27:27,447 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-01 17:27:27,448 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
894,0|2372,1|3463,2|1020,3|909,4|1048,5|790,6|617,7|581,8|614,9|712,10|823,11|614,12|591,13|704,14|919,15|805,16|3402,17|
[ 894. 2372. 3463. 1020.  909. 1048.  790.  617.  581.  614.  712.  823.
  614.  591.  704.  919.  805. 3402.]
2023-05-01 17:27:36,137 | INFO     |: Update Step
2023-05-01 17:27:36,138 | INFO     |:      | Update Time: 0.001 secs
2023-05-01 17:27:36,139 | INFO     |: Bel index     : (2, 2, 10) with prob = 1.0
2023-05-01 17:27:36,139 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2023-05-01 17:27:36,140 | INFO     |: Belief        : (-0.914, -0.610, 30.000)


In [11]:
ble.disconnect()

2023-05-01 17:33:31,510 | INFO     |: Disconnected from C0:83:B1:6A:38:3C
